In [29]:
# !pip install mediapipe



INSTALLING DEPENDENCEY

In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd
import time
import mediapipe as mp

In [2]:
import mediapipe as mp
print(mp.__version__)


0.10.14


Using Mediapipe

In [2]:
mp_holi=mp.solutions.holistic #models in mp
mp_draw=mp.solutions.drawing_utils # drawing Function

In [3]:
def media_dect(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [4]:
def draw_holist(image,results):
    mp_draw.draw_landmarks(image,results.face_landmarks,mp_holi.FACEMESH_CONTOURS,
                          mp_draw.DrawingSpec(color=(144,238,144),thickness=1,circle_radius=1),
                          mp_draw.DrawingSpec(color=(127,255,212),thickness=1 ,circle_radius=1)
                          )
    mp_draw.draw_landmarks(image,results.pose_landmarks,mp_holi.POSE_CONNECTIONS,
                          mp_draw.DrawingSpec(color=(10,10,100),thickness=1,circle_radius=4),
                          mp_draw.DrawingSpec(color=(100,10,10),thickness=1,circle_radius=2)
                          )
    mp_draw.draw_landmarks(image,results.left_hand_landmarks,mp_holi.HAND_CONNECTIONS,

                          mp_draw.DrawingSpec(color=(10,10,100),thickness=2,circle_radius=4),
                          mp_draw.DrawingSpec(color=(100,10,10),thickness=2,circle_radius=2))
    mp_draw.draw_landmarks(image,results.right_hand_landmarks,mp_holi.HAND_CONNECTIONS,
                          mp_draw.DrawingSpec(color=(10,10,100),thickness=2,circle_radius=4),
                          mp_draw.DrawingSpec(color=(100,10,10),thickness=2,circle_radius=2))

In [27]:


cap = cv2.VideoCapture(0)
cap.set(1200,640)
cap.set(1200,480)

with mp_holi.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holi:
    while cap.isOpened():
        ret, frame=cap.read()
    
        image,results=media_dect(frame,holi)
        # print(results)
        draw_holist(image,results)
        cv2.imshow("frame",image)
        if cv2.waitKey(10) & 0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

C:\Users\munna\anaconda3_\envs\sign\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [7]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
# plt.imshow(frame)

In [39]:
len(results.right_hand_landmarks.landmark)

21

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


In [9]:
pwd

'C:\\Users\\munna\\sign_alnguage'

In [6]:
import os
import numpy as np

# Define paths and parameters
data_path = os.path.join('data')
actions = np.array(['hello', 'thanks', 'i love U'])
no_sequence = 30
sequence_length = 30
start_folder = 1

In [38]:


# Ensure the base data path exists
os.makedirs(data_path, exist_ok=True)

for action in actions:
    action_path = os.path.join(data_path, action)
    os.makedirs(action_path, exist_ok=True)
    
    try:
        dirmax = np.max(np.array(os.listdir(action_path)).astype(int))
    except ValueError:  # Raised if the directory is empty
        dirmax = 0
    
    for sequence in range(1, no_sequence + 1):
        sequence_path = os.path.join(action_path, str(dirmax + sequence))
        os.makedirs(sequence_path, exist_ok=True)


In [8]:
for action in actions:
    print(action)

hello
thanks
i love U


In [40]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holi.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequence):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = media_dect(frame, holistic)

                # Draw landmarks
                draw_holist(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(data_path, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

C:\Users\munna\anaconda3_\envs\sign\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [57]:
cap.release()
cv2.destroyAllWindows()

In [7]:

label_map = {label:num for num, label in enumerate(actions)}

In [8]:
label_map

{'hello': 0, 'thanks': 1, 'i love U': 2}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(data_path, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            print(os.path.join(data_path, action, str(sequence),"{}.npy".format(frame_num)))
            res = np.load(os.path.join(data_path, action, str(sequence), "{}.npy".format(frame_num)),allow_pickle=True)
            window.append(res)
            # print(res)
        sequences.append(window)
        labels.append(label_map[action])

data\hello\1\0.npy
data\hello\1\1.npy
data\hello\1\2.npy
data\hello\1\3.npy
data\hello\1\4.npy
data\hello\1\5.npy
data\hello\1\6.npy
data\hello\1\7.npy
data\hello\1\8.npy
data\hello\1\9.npy
data\hello\1\10.npy
data\hello\1\11.npy
data\hello\1\12.npy
data\hello\1\13.npy
data\hello\1\14.npy
data\hello\1\15.npy
data\hello\1\16.npy
data\hello\1\17.npy
data\hello\1\18.npy
data\hello\1\19.npy
data\hello\1\20.npy
data\hello\1\21.npy
data\hello\1\22.npy
data\hello\1\23.npy
data\hello\1\24.npy
data\hello\1\25.npy
data\hello\1\26.npy
data\hello\1\27.npy
data\hello\1\28.npy
data\hello\1\29.npy
data\hello\10\0.npy
data\hello\10\1.npy
data\hello\10\2.npy
data\hello\10\3.npy
data\hello\10\4.npy
data\hello\10\5.npy
data\hello\10\6.npy
data\hello\10\7.npy
data\hello\10\8.npy
data\hello\10\9.npy
data\hello\10\10.npy
data\hello\10\11.npy
data\hello\10\12.npy
data\hello\10\13.npy
data\hello\10\14.npy
data\hello\10\15.npy
data\hello\10\16.npy
data\hello\10\17.npy
data\hello\10\18.npy
data\hello\10\19.npy


In [41]:
res=np.load(os.path.join('data','hello','11','2.npy'),allow_pickle=True)

In [45]:

np.array(labels).shape

(90,)

In [15]:
X=np.array(sequences)

In [47]:
X.shape

(90, 30, 1662)

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
y=to_categorical(labels).astype(int)

NameError: name 'labels' is not defined

In [ ]:
y

In [16]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [17]:
x_test.shape

(9, 30, 1662)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [12]:

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [13]:

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

C:\Users\munna\anaconda3_\envs\sign\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
x_train.shape

(81, 30, 1662)

In [23]:
y_train.shape

(81, 3)

In [ ]:

model.fit(x_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - categorical_accuracy: 0.3424 - loss: 1.0916
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - categorical_accuracy: 0.4807 - loss: 1.3625
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.3408 - loss: 1.0933
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - categorical_accuracy: 0.3772 - loss: 1.0310
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - categorical_accuracy: 0.4007 - loss: 1.0249
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - categorical_accuracy: 0.4790 - loss: 0.9555
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - categorical_accuracy: 0.5037 - loss: 0.8573
Epoch 8/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - categorical_accuracy: 0.6946 - loss: 0.8209
Epoch 9/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - categorical_accuracy: 0.6479 - loss: 1.4233
Epoch 10/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - categorical_accuracy: 0.5919 - loss: 0.9631
Epoch 11/1000
3/3 ━━━━━━━━━━━

In [ ]:
model.save('action.h5')

In [ ]:

del model


In [16]:
model.load_weights('action.h5')


In [19]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holi.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = media_dect(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_holist(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

C:\Users\munna\anaconda3_\envs\sign\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

NameError: name 'prob_viz' is not defined